In [1]:
import os
import pandas as pd

In [2]:
ALPHAVANTAGE_KEY = os.getenv('ALPHAVANTAGE_KEY')

In [3]:
URL = 'https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol=BTC&market=USD&apikey=%s&datatype=csv'.format(ALPHAVANTAGE_KEY)

In [4]:
def total_return(prices):
  return prices.iloc[-1] / prices.iloc[0] - 1

In [5]:
df = pd.read_csv(URL)
df['timestamp'] = pd.to_datetime(df.timestamp)
df.index = df.timestamp.dt.date
df = df[::-1]
df.rename(columns={"close (USD)": "btc_close"}, inplace=True)
df = df.drop(['open (USD)', 'high (USD)', 'low (USD)', 'open (USD).1', 'high (USD).1', 'low (USD).1', 'close (USD).1'], axis=1)
df['pct_change'] = df['btc_close'].pct_change()
df[['timestamp', 'btc_close', 'pct_change']].to_csv('bitcoin_daily_returns.csv')
df[['timestamp', 'btc_close', 'pct_change']]
df

,timestamp,btc_close,volume,market cap (USD),pct_change
timestamp,,,,,
2020-02-06,2020-02-06,9772.00,64949.706588,64949.706588,NaN
2020-02-07,2020-02-07,9813.73,43966.114632,43966.114632,0.004270
2020-02-08,2020-02-08,9895.05,43600.843666,43600.843666,0.008286
2020-02-09,2020-02-09,10151.75,43408.475616,43408.475616,0.025942
2020-02-10,2020-02-10,9851.83,59573.084619,59573.084619,-0.029544
...,...,...,...,...,...
2022-10-28,2022-10-28,20591.84,287039.945690,287039.945690,0.014621
2022-10-29,2022-10-29,20809.67,254881.777550,254881.777550,0.010578
2022-10-30,2022-10-30,20627.48,192795.608860,192795.608860,-0.008755


In [6]:
btc_monthly_returns = df.groupby([df.timestamp.dt.year, df.timestamp.dt.month])['btc_close'].apply(total_return)
btc_monthly_returns.to_csv("bitcoin_monthly_returns.csv")
btc_monthly_returns

timestamp  timestamp
2020       2           -0.127752
           3           -0.248649
           4            0.297622
           5            0.070388
           6           -0.104131
           7            0.227844
           8           -0.012851
           9           -0.096073
           10           0.298694
           11           0.431230
           12           0.541364
2021       1            0.128233
           2            0.346273
           3            0.184595
           4           -0.017476
           5           -0.355475
           6           -0.044916
           7            0.237493
           8            0.182090
           9           -0.102159
           10           0.273323
           11          -0.065022
           12          -0.191787
2022       1           -0.193949
           2            0.115401
           3            0.024518
           4           -0.186950
           5           -0.173319
           6           -0.330929
           7          